### Cargamos herramientas necesarias

In [72]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns

#from pandas import ExcelWriter
#import itertools

pd.set_option('display.max_columns', 200) 

df_2013 = pd.read_csv('../datos/datos-2013.csv', index_col = 0, encoding='ISO-8859-1', sep=';')
df_2014 = pd.read_csv('../datos/datos-2014.csv', index_col = 0, encoding='ISO-8859-1', sep=';')
df_2015 = pd.read_csv('../datos/datos-2015.csv', index_col = 0, encoding='ISO-8859-1', sep=';')
df_2016 = pd.read_csv('../datos/datos-2016.csv', index_col = 0, encoding='ISO-8859-1', sep=';')
df_2017 = pd.read_csv('../datos/datos-2017.csv', index_col = 0, encoding='ISO-8859-1', sep=';')
df_2018 = pd.read_csv('../datos/datos-2018.csv', index_col = 0, encoding='ISO-8859-1', sep=';')
df_2019 = pd.read_csv('../datos/datos-2019.csv', index_col = 0, encoding='ISO-8859-1', sep=';')
df_2020 = pd.read_csv('../datos/datos-2020.csv', index_col = 0, encoding='ISO-8859-1', sep=';')
df_2021 = pd.read_csv('../datos/datos-2021.csv', index_col = 0, encoding='ISO-8859-1', sep=';')

# Análisis individual de los datasets

Realizamos una inspección previa a la unión de los mismos para ver que el númeor de columnas y nombre no difiere.

In [ ]:
print(df_2013.shape)
df_2013.head(2)

In [ ]:
print(df_2014.shape)
print(df_2014.columns)
df_2014.head(2)

In [ ]:
print(df_2015.shape)
print(df_2015.columns)
df_2015.head(2)

In [ ]:
print(df_2016.shape)
print(df_2016.columns)
df_2016.head(2)

In [ ]:
print(df_2017.shape)
print(df_2017.columns)
df_2017.head(2)

In [ ]:
print(df_2018.shape)
print(df_2018.columns)
df_2018.head(2)

In [ ]:
print(df_2019.shape)
print(df_2019.columns)
df_2019.head(2)

In [ ]:
print(df_2020.shape)
print(df_2020.columns)
df_2020.head(2)

In [ ]:
print(df_2021.shape)
print(df_2021.columns)
df_2021.head(2)